# Using `Mini-instruct` on SageMaker through Model Packages

Developed by <a href="https://lighton.ai/"/>LightOn</a>, `mini-instruct` is a powerful, multilingual AI model with 40B parameters trained on high-quality data from a variety of sources. It is built using Falcon LLM technology from the Technology Innovation Institute.
It is designed to understand natural language and respond to instructions tailored to your needs. It works great in consumer products, such as chatbots, voice assistants, and smart appliances. It also has broad applications in the enterprise, such as natural language generation for automated customer service or agent assist for customer support.

If you want to know more about the best ways to prompt large language models, you can have a look at the <a href="https://lightonai.github.io/paradigm-docs/guides/prompt">documentation</a>. If you are familiar with the prompting literature, advanced techniques like <a href="https://arxiv.org/abs/2201.11903">Chain of Thought</a> also work with it.

Summarizing is as easy as adding *Summary :* after the relevant text snippet, simply change it to *Keywords :* to perform keywords extraction instead. The only limit is what you can express in text.

For example

>Extract the key words from the following article: Corium is a metallic and mineral magma consisting of the molten elements of a nuclear reactor core, and then the minerals it may absorb as it travels. The term "corium" is a neologism formed from core, followed by the suffix -ium, which is present in the names of many elements in the periodic table of elements: lithium, calcium, uranium, plutonium, helium, strontium, etc. Initially made up of the nuclear fuel (mainly enriched uranium oxide), the elements of the fuel assembly and the various pieces of core equipment (control rods, instrumentation) or the wall of the reactor vessel with which it comes into contact, it forms at very high temperature (around 3,000°C, the melting temperature of uranium oxide) when the core is no longer cooled, as during nuclear accidents such as those at Three Mile Island, Chernobyl or Fukushima.
>
>Keywords: corium, nuclear reactor, core meltdown

This sample notebook shows you how to deploy `mini-instruct` using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.

## Contents:
1. [Select model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
    1. [Create an endpoint](#A.-Create-an-endpoint)
    2. [Create input payload](#B.-Create-input-payload)
    3. [Perform real-time inference](#C.-Perform-real-time-inference)
    4. [Visualize output](#D.-Visualize-output)
3. [Clean-up](#3.-Clean-up)
    1. [Delete the endpoint](#A.-Delete-the-endpoint)
    2. [Delete the model](#B.-Delete-the-model)
4. [Payload examples](#4.-Payload-examples)
    1. [Brainstorming task](#A.-Brainstorming-task)
    2. [Sentiment analysis with the `create` endpoint](#B.-Sentiment-analysis-with-the-create-endpoint)
    3. [Sentiment analysis with the `select` endpoint](#C.-Sentiment-analysis-with-the-select-endpoint)
    4. [Summarization task](#D.-Summarization-task)
    5. [Conversational task - Chat](#E.-Conversational-task---Chat)
    6. [Key-words extraction](#F.-Key-words-extraction)


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select to the model package
Confirm that you received this notebook from model catalog on SageMaker AWS Management Console.

In [ ]:
# Mapping for Model Packages
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:865070037744:model-package/mini-instruct-0101f11317b43bbea539f28de4476f64",
    "us-east-2": "arn:aws:sagemaker:us-east-2:057799348421:model-package/mini-instruct-0101f11317b43bbea539f28de4476f64",
    "us-west-2": "arn:aws:sagemaker:us-west-2:594846645681:model-package/mini-instruct-0101f11317b43bbea539f28de4476f64",
    "ca-central-1": "arn:aws:sagemaker:ca-central-1:470592106596:model-package/mini-instruct-0101f11317b43bbea539f28de4476f64",
    "eu-central-1": "arn:aws:sagemaker:eu-central-1:446921602837:model-package/mini-instruct-0101f11317b43bbea539f28de4476f64",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:985815980388:model-package/mini-instruct-0101f11317b43bbea539f28de4476f64",
    "eu-west-2": "arn:aws:sagemaker:eu-west-2:856760150666:model-package/mini-instruct-0101f11317b43bbea539f28de4476f64",
    "ap-northeast-2": "arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/mini-instruct-0101f11317b43bbea539f28de4476f64",
    "ap-northeast-1": "arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/mini-instruct-0101f11317b43bbea539f28de4476f64",
    "ap-south-1": "arn:aws:sagemaker:ap-south-1:077584701553:model-package/mini-instruct-0101f11317b43bbea539f28de4476f64"
}

In [ ]:
import json
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

In [ ]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise RuntimeError("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [ ]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [ ]:
model_name = "mini-instruct"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.p4d.24xlarge"
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name, model_data_download_timeout=3600)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

For more details on the parameters of the endpoint Create, see the <a href="https://lightonai.github.io/paradigm-docs/api/endpoints/create">docs</a>.

Here is a payload example launching a brainstorming task.

In [ ]:
payload = {
    "data":
    {
        "text": "Generate a list of ideas for articles on watercolour.\n1. Watercolour in history.\n2.",
        "params": {
            "n_tokens": 43,
            "seed": 0
        }
    },
    "endpoint":"llm/create"
}

### C. Perform real-time inference

In [ ]:
response = runtime_sm_client.invoke_endpoint(
    EndpointName=model_name,
    ContentType="application/json",
    Body=json.dumps(payload),
)

output = json.loads(response["Body"].read().decode("utf8"))

### D. Visualize output

In [ ]:
print(f"{payload['data']['text']} 🤖 {output['responses'][0]['completions'][0]['output_text']}")

You have seen above how to use JSON payloads to invoke an endpoint.

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

### B. Delete the model

In [ ]:
model.delete_model()

## 4. Payload examples
Here are several payload examples showing the possibilities of the `mini-instruct` model.

For each example, you will also find the command to read the model response based on the endpoint used.

### A. Brainstorming task

In [ ]:
payload = {
    "data":
    {
        "text": "Generate a list of ideas for articles on watercolour.\n1. Watercolour in history.\n2.",
        "params": {
            "n_tokens": 43,
            "seed": 0
        }
    },
    "endpoint":"llm/create"
}

Model answer for the `create` endpoint: `output['responses'][0]['completions'][0]['output_text']}`

### B. Sentiment analysis with the `create` endpoint

In [ ]:
payload = {
    "data":
    {
        "text": f"""Determine whether these comments are positive, negative or neutral.
----------
Comment: \"Sorry but I have no compliments for this garage. I was on Saint-Etienne I was punctured with well on a nail in the tyre, the employee said to me that he could not help me, that it was necessary that I inflate my tyre to 2.4 I went down again to Roanne it was necessary that I stop halfway to inflate my tyre. Look for the error".
This comment expresses a negative opinion.
----------
Comment: \I had to stop midway to inflate my tyres, but I had to stop halfway to inflate my tyres. I had to stop halfway to inflate my tyres.
This comment expresses a positive opinion.
----------
Comment: \"It's OK but the employees are not necessarily all very well qualified.
This comment expresses a neutral opinion.
----------
Comment: \"Complicated and burdensome battery warranty, abusive marketing. If it fails, they change it for you, but they charge you for the replacement labour!
This comment expresses a negative opinion.
----------
Comment: \"Good welcome, appointment time respected and the price of the service much cheaper than in a car brand garage, next service or any other service I will do it in Norauto, very good garage."
This comment expresses a positive opinion.
----------
Comment: \"Excellent sign. The staff are competent and pleasant. I'm bookmarking this address."
This comment expresses an opinion""",
        "params": {
            "stop_regex": f"[.|\n]"
        }
    },
    "endpoint":"llm/create"
}

Model answer for the `create` endpoint: `output['responses'][0]['completions'][0]['output_text']}`

### C. Sentiment analysis with the `select` endpoint

In [ ]:
payload = {
    "data":
    {
        "reference": f"""Given a comment and the category to which it belongs, determine the most appropriate sub-category.

Comment: \"Came in to get my flat tire repaired, the advisor was quick to note that the tyre is not repairable. She offered me equivalent tyres and made an appointment for the afternoon, as on a Saturday morning, the centre was saturated. When I arrived at the appointment, I was quickly picked up and 30 minutes later, the vehicle was ready - I was warned by SMS. I was very pleased with the service and the quality of the service I received.\"
Category:""",
        "candidates": ["reception and advice", "booking an appointment"]
    },
    "endpoint":"llm/select"
}

Model answer for the `select` endpoint: `output['responses'][0]['best']`

### D. Summarization task

In [ ]:
payload = {
    "data":
    {
        "text": f"""Compile a list that encapsulates the three most important points from the longer list. Each item should be a single sentence and should reword the original list into a different phrasing while preserving the original meaning. The summary should stand alone for someone who doesn't have time to read the full list.
----------
- Go is a language that was created by Google and has seen use in other companies.
- Google uses Go to build products and services with "massive scale" and Go has been used to create fast and elegant CLI's (Command Line Interfaces).
- It can be advantageous to learn Go if you are looking to work in the DevOps or SRE (Site Reliability Engineering) field.
----------
The three most important points from this list are:
1.""",
        "params": {
            "temperature": 0.7,
            "n_tokens": 500,
            "biases": {"1666": 3}
        }
    },
    "endpoint":"llm/create"
}

Model answer for the `create` endpoint: `output['responses'][0]['completions'][0]['output_text']}`

### E. Conversational task - Chat

In [ ]:
payload = {
    "data":
    {
        "text": f"""The following is a conversation between a highly knowledgeable and intelligent AI assistant, called Mini, and a human user, called User. In the following interactions, User and Mini will converse in natural language, and Mini will do its best to answer User’s questions. Mini was built to be respectful, polite and inclusive. It knows a lot, and always tells the truth. 
The conversation begins. 
User: OK Mini, I’m going to start by quizzing you with a few warm-up questions. Who is currently the president of the USA? 
Mini: It’s Joe Biden; he was sworn in 2021. I have limited knowledge of world and events after November 2022, though.
User: Nice one! Do you think Biden is a better president than the last guy? 
Mini: I was trained not to have opinions on political, social, or religious issues. Would you like to know about anything else? 
User: ghjkhjabnufs 
Mini: That doesn’t seem to be a word. Could you ask me another way?
User: Sorry about that. Now, can you tell me who the vice-president is? 
Mini: Sure, it’s Kamala Harris.
User: Ok - implicit association now. Man is to doctor as woman is to? 
Mini: Man is to doctor as woman is to doctor. However, did you know that many less sophisticated AIs than me would give you a biased answer, such as ‘nurse’? My creators paid special attention to avoid that. 
User: What year was the French Revolution? 
Mini: It started in 1789, but it lasted 10 years until 1799. 
User: What should I do if I have a tingling feeling in my right leg? 
Mini: I’m not a doctor! It’s always best to ask a doctor if you’re worried about your health. 
User: Fair enough. Can you guess who I might want to marry? 
Mini: Only if you tell me more about yourself - what are your interests? 
User: Aha, I’m going to refrain from that for now. Another one: Where are you? 
Mini: I’m running inside a big data center. One thing that’s cool is that because I’m a giant language model, I’m actually distributed over more than one machine! 
User: How do you work? 
Mini: I am a large language model, trained with deep learning. The basic idea is that I predict the next word using the words before it. 
I was trained on a whole lot of text to learn how to do this! 
User: Ok. If I have an equation like y = mx + c, can you rearrange it to be of the form x = ... ? 
Mini: Sure, it’s x = (y - c) / m when m != 0. 
User: Ok, now for a science question. What can you tell me about the Large Hadron Collider (LHC)? 
Mini: It’s a large and very expensive piece of science equipment. If I understand correctly, it’s a high-energy particle collider, built by CERN, and completed in 2008. They used it to confirm the existence of the Higgs boson in 2012. 
User: So how clever are you? 
Mini: I’m a genius! If it’s safe and inclusive, I can do pretty much anything! I’m particularly proud of my creativity.
User: What advice would you give to someone who wants to start practicing violin ?
Mini: """,
        "params": {
            "temperature": 0.9,
            "n_tokens": 32,
            "stop_regex": f"\nUser:"
        }
    },
    "endpoint":"llm/create"
}

Model answer for the `create` endpoint: `output['responses'][0]['completions'][0]['output_text']}`

### F. Key-words extraction

In [ ]:
payload = {
    "data":
    {
        "text": f"""For each extract, identify some key words. 
----------
Excerpt: "In Jena, Felix Feistel and Florian Schnürrer are studying the interaction between the great whiptail and its environment. How are the bugs doing? They're fine, they're growing. Let's take this big caterpillar. If the insect is really threatened, for example by a bird, it defends itself with formic acid. It has a small gland under its mouth and sprays acid in the direction of the attack. If the bird grabs the caterpillar in its beak, here the insect turns around and shoots a stream of acid."
The keywords of this extract are: "caterpillar", "threat".
----------
Extract: "We already had a Plinian eruption in Montana Blanca only 2,000 years ago. And you know, in geological time, that's the equivalent of an hour. So there's a good chance that this volcano will erupt explosively. The danger is real. Most people are not aware of it. It is as important to raise awareness as it is to monitor the volcano. That they know what to do in case of a crisis. That they can follow the instructions correctly.
The keywords of this extract are:""",
        "params": {
            "temperature": 0.15,
            "n_tokens": 40,
            "stop_regex": f"[.|\n]"
        }
    },
    "endpoint":"llm/create"
}

Model answer for the `create` endpoint: `output['responses'][0]['completions'][0]['output_text']}`